In [85]:
import os, sys, warnings
import concurrent.futures
from openpyxl import Workbook
from multiprocessing import Pool
from pathlib import Path
from IPython.display import Image, display
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
#sys.path.insert(0, os.path.abspath(os.path.join(Path.home(), "rpa_libs")))

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys

# Nossas libs
#import app
#from plugins.selenium.selenium import Selenium

ModuleNotFoundError: No module named 'openpyxl'

In [58]:
# Exemplo de uso do arquivo de configuração
#app.redefine_config('caminho do seu arquivo .yaml')
#teste
import pandas as pd

In [59]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [66]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [67]:
# Def the website i want to extract the table for
url = 'https://www.coingecko.com/pt'
driver.get(url)

In [68]:
# Picking table from Cripto Coins site and creating a variable for the rows in this table
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [69]:
# Creating the function to iterate in rows and cells of the table and appending to the table_data
def process_cells(cells):
    cell_data = []
    for cell in cells:
        img_tag = cell.find_elements(By.TAG_NAME, 'img')
        if img_tag:
            # Extract the image URL (src attribute)
            img_src = img_tag[0].get_attribute('src')
            cell_data.append(img_src)
        if '$' in cell.text:
            cell_splited = cell.text.split(" ")
            for part_splitted in cell_splited:
                cell_data.append(part_splitted)
        else: cell_data.append(cell.text)
    #table_data.append(cell_data)
    return cell_data

In [70]:
#with concurrent.futures.ThreadPoolExecutor() as executor:
#    results = list(executor.map(process_rows, rows))

#with Pool() as pool:
#    result = pool.map(process_rows, rows)

In [71]:
def process_rows(rows):
    table_data = []
    #for row in rows:
    for x in range(10):
        row = rows[x]
        cells = row.find_elements(By.XPATH, './/td')
        cells_data = process_cells(cells)
        table_data.append(cells_data)
    return table_data

In [72]:
table_data = process_rows(rows)

In [73]:
list(table_data)

[['',
  '1',
  'https://assets.coingecko.com/coins/images/1/standard/bitcoin.png?1696501400',
  'Bitcoin\nBTC',
  'Comprar',
  'US$',
  '63.446,99',
  '0.4%',
  '0.0%',
  '6.6%',
  '',
  'US$',
  '28.605.925.824',
  'US$',
  '1.253.639.173.509',
  '',
  '',
  'https://www.coingecko.com/coins/1/sparkline.svg',
  ''],
 ['',
  '2',
  'https://assets.coingecko.com/coins/images/279/standard/ethereum.png?1696501628',
  'Ethereum\nETH',
  'Comprar',
  'US$',
  '2.598,89',
  '0.2%',
  '1.3%',
  '13.1%',
  '',
  'US$',
  '14.118.293.089',
  'US$',
  '312.857.111.944',
  '',
  '',
  'https://www.coingecko.com/coins/279/sparkline.svg',
  ''],
 ['',
  '3',
  'https://assets.coingecko.com/coins/images/325/standard/Tether.png?1696501661',
  'Tether\nUSDT',
  'Comprar',
  'US$',
  '0,9994',
  '0.2%',
  '0.1%',
  '0.2%',
  '',
  'US$',
  '45.055.024.764',
  'US$',
  '119.124.974.151',
  '',
  '',
  'https://www.coingecko.com/coins/325/sparkline.svg',
  ''],
 ['',
  '4',
  'https://assets.coingecko.com

In [74]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
headers = [item for item in headers if item]
headers[0] = "IMG Coin"
headers.insert(0, "ID")
headers.insert(3, "$")
headers.insert(8, "$")
headers.insert(10, "$")
#table_data = [[cell for cell in row if cell.strip()] for row in table_data]

In [75]:
list(headers)

['ID',
 'IMG Coin',
 'Moeda',
 '$',
 'Preço',
 '1 h',
 '24 h',
 '7 d',
 '$',
 'Volume em 24 h',
 '$',
 'Capitalização de mercado',
 'Últimos 7 dias']

In [76]:
list(table_data)

[['',
  '1',
  'https://assets.coingecko.com/coins/images/1/standard/bitcoin.png?1696501400',
  'Bitcoin\nBTC',
  'Comprar',
  'US$',
  '63.446,99',
  '0.4%',
  '0.0%',
  '6.6%',
  '',
  'US$',
  '28.605.925.824',
  'US$',
  '1.253.639.173.509',
  '',
  '',
  'https://www.coingecko.com/coins/1/sparkline.svg',
  ''],
 ['',
  '2',
  'https://assets.coingecko.com/coins/images/279/standard/ethereum.png?1696501628',
  'Ethereum\nETH',
  'Comprar',
  'US$',
  '2.598,89',
  '0.2%',
  '1.3%',
  '13.1%',
  '',
  'US$',
  '14.118.293.089',
  'US$',
  '312.857.111.944',
  '',
  '',
  'https://www.coingecko.com/coins/279/sparkline.svg',
  ''],
 ['',
  '3',
  'https://assets.coingecko.com/coins/images/325/standard/Tether.png?1696501661',
  'Tether\nUSDT',
  'Comprar',
  'US$',
  '0,9994',
  '0.2%',
  '0.1%',
  '0.2%',
  '',
  'US$',
  '45.055.024.764',
  'US$',
  '119.124.974.151',
  '',
  '',
  'https://www.coingecko.com/coins/325/sparkline.svg',
  ''],
 ['',
  '4',
  'https://assets.coingecko.com

In [77]:
df = pd.DataFrame(table_data)
df_treated = df.loc[:, (df != '').any(axis=0)]
df_treated = df_treated.drop(df.columns[4], axis=1)


In [78]:
df_treated

,1,2,3,5,6,7,8,9,11,12,13,14,17
0,1,https://assets.coingecko.com/coins/images/1/st...,Bitcoin\nBTC,US$,"63.446,99",0.4%,0.0%,6.6%,US$,28.605.925.824,US$,1.253.639.173.509,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/...,Ethereum\nETH,US$,"2.598,89",0.2%,1.3%,13.1%,US$,14.118.293.089,US$,312.857.111.944,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/...,Tether\nUSDT,US$,"0,9994",0.2%,0.1%,0.2%,US$,45.055.024.764,US$,119.124.974.151,https://www.coingecko.com/coins/325/sparkline.svg
3,4,https://assets.coingecko.com/coins/images/825/...,BNB\nBNB,US$,"592,74",0.3%,2.5%,10.1%,US$,877.955.378,US$,86.532.056.517,https://www.coingecko.com/coins/825/sparkline.svg
4,5,https://assets.coingecko.com/coins/images/4128...,Solana\nSOL,US$,"151,03",0.2%,2.7%,18.5%,US$,3.401.606.505,US$,70.869.108.525,https://www.coingecko.com/coins/4128/sparkline...
5,6,https://assets.coingecko.com/coins/images/6319...,USDC\nUSDC,US$,"0,999",0.2%,0.2%,0.2%,US$,6.943.096.671,US$,36.035.463.081,https://www.coingecko.com/coins/6319/sparkline...
6,7,https://assets.coingecko.com/coins/images/44/s...,XRP\nXRP,US$,"0,586",0.2%,0.1%,3.3%,US$,1.210.945.674,US$,33.078.006.648,https://www.coingecko.com/coins/44/sparkline.svg
7,8,https://assets.coingecko.com/coins/images/1344...,Lido Staked Ether\nSTETH,US$,"2.598,21",0.2%,1.3%,12.9%,US$,31.912.060,US$,25.433.474.759,https://www.coingecko.com/coins/13442/sparklin...
8,9,https://assets.coingecko.com/coins/images/5/st...,Dogecoin\nDOGE,US$,"0,1097",0.5%,1.5%,9.0%,US$,784.202.137,US$,16.039.414.158,https://www.coingecko.com/coins/5/sparkline.svg
9,10,https://assets.coingecko.com/coins/images/1798...,Toncoin\nTON,US$,"5,68",0.3%,2.8%,4.5%,US$,419.847.577,US$,14.420.029.001,https://www.coingecko.com/coins/17980/sparklin...


In [36]:
df_treated

,ID,IMG Coin,Moeda,Preço,1 h,24 h,7 d,Volume em 24 h,Capitalização de mercado,Últimos 7 dias
0,1,https://assets.coingecko.com/coins/images/1/st...,Bitcoin\nBTC,"US$ 63.709,57",0.0%,0.8%,7.1%,US$ 29.159.998.316,US$ 1.258.824.475.913,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/...,Ethereum\nETH,"US$ 2.617,67",0.0%,0.1%,14.1%,US$ 14.518.629.149,US$ 315.093.786.066,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/...,Tether\nUSDT,"US$ 0,9989",0.2%,0.3%,0.2%,US$ 47.387.813.918,US$ 119.030.268.021,https://www.coingecko.com/coins/325/sparkline.svg
3,4,https://assets.coingecko.com/coins/images/825/...,BNB\nBNB,"US$ 597,40",0.2%,0.9%,10.5%,US$ 960.188.515,US$ 87.161.257.215,https://www.coingecko.com/coins/825/sparkline.svg
4,5,https://assets.coingecko.com/coins/images/4128...,Solana\nSOL,"US$ 150,98",0.3%,3.5%,17.3%,US$ 3.426.505.529,US$ 70.741.430.083,https://www.coingecko.com/coins/4128/sparkline...
5,6,https://assets.coingecko.com/coins/images/6319...,USDC\nUSDC,"US$ 0,9988",0.2%,0.2%,0.1%,US$ 6.804.098.999,US$ 35.919.101.690,https://www.coingecko.com/coins/6319/sparkline...
6,7,https://assets.coingecko.com/coins/images/44/s...,XRP\nXRP,"US$ 0,5889",0.2%,0.6%,3.6%,US$ 1.277.691.214,US$ 33.236.388.297,https://www.coingecko.com/coins/44/sparkline.svg
7,8,https://assets.coingecko.com/coins/images/1344...,Lido Staked Ether\nSTETH,"US$ 2.618,61",0.1%,0.3%,14.4%,US$ 33.640.987,US$ 25.639.480.674,https://www.coingecko.com/coins/13442/sparklin...
8,9,https://assets.coingecko.com/coins/images/5/st...,Dogecoin\nDOGE,"US$ 0,1094",0.3%,1.4%,9.2%,US$ 763.691.274,US$ 15.990.003.282,https://www.coingecko.com/coins/5/sparkline.svg
9,10,https://assets.coingecko.com/coins/images/1798...,Toncoin\nTON,"US$ 5,69",0.1%,3.1%,3.2%,US$ 430.046.636,US$ 14.432.717.350,https://www.coingecko.com/coins/17980/sparklin...


In [385]:
#df.style
#df.style.highlight_max()
#df.style.highlight_null(null_color="green")
#df.style.set_na_rep("OutofScope").highlight_null(null_color="orange")
#df.style.background_gradient()
#df.style.set_properties(**{'border': '1.3px solid black',
 #                         'color': 'white'})
#df.style.hide_index()

list(df_treated)

In [79]:
df_treated.style\
    .highlight_max(subset=[5], color='green')\
    .background_gradient(cmap='viridis', subset=[4])\
    .set_properties(**{'border': '1.3px solid black', 'color': 'white'})
df_treated = df_treated.reset_index(drop=True)

In [27]:
#df_treated

In [81]:
df_treated.columns = [headers]


In [84]:
df_treated.to_excel("file_test_df_cripto_coin.xlsx")

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    table_data.append([cell.text for cell in cells])

In [ ]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    table_data.append([cell.text for cell in cells])